In [1]:
## Import necessary packages
# On Macs use osx
# For Windows use qt
%matplotlib qt
import ffmpeg
import numpy as np
from numpy.random import rand
from landscapeWithOcean import LandscapeWithOcean # Import methods from inside file landscapeWithOcean.py

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm

In [3]:
def AddHill(Z,NX,NY,xx,yy,r,h):

    for x in range(NX):
        for y in range(NY):
            dx = np.mod(x-xx+NX/2,NX)-NX/2; # difference i-i0 but apply p.b.c. 
            dy = np.mod(y-yy+NY/2,NY)-NY/2;
            dr = np.sqrt(dx**2+dy**2);
            if (dr<r):
                Z[x,y] += h * (np.cos(dr/r*np.pi/2.0))**2;
    return Z

In [4]:
### Define simulation grid and initial conditions

NX = 35*2 #number of rows
NY = 35*2 #number of columns

d  = 5 # grid spacing in meters
dx = d # keep dx=dy for simplicity
dy = d

LX=NX*dx
LY=NY*dy

# small random features in topography to begin erosion
Z = rand(NX,NY)*0.1
for i in range(5):
    xx = rand()*NX
    yy = rand()*NY
    r  = (0.1+rand())*NX
    h  = (0.1+rand())*5
    Z = AddHill(Z,NX,NY,xx,yy,r,h)

for i in range(5):
    xx = rand()*NX
    yy = rand()*NY
    r  = (0.1+rand())*NX/2
    h  = (0.5+rand())*10
    Z = AddHill(Z,NX,NY,xx,yy,r,h)

x = np.arange(NX)
y = np.arange(NY)
X,Y = np.meshgrid(y,x) #strange that y goes first !!!
ZMaxOrg = np.max(Z)
#print(ZMaxOrg)

In [5]:
### Physical Parameters
K = 1.0e-6 # meters^(1-2m)/yr

D = 0.005 # m^2/yr

# uplift rate
# uplift = 0.03 / 600.
uplift = 0.0

In [6]:
### Model parameters

# Time step
dt = d**2 / D / 8. 
#dt = d**2 / D /16. #extra small steps 
print(' dt[years] = ',dt)

#Area exponent A^m, default m=1
m=4

#gradient exponent g^n, default n=1
n=1

#erosion threshold 
theta_c = 10 

 dt[years] =  625.0


In [7]:
# Total simulation time
T = 1000.0 * 625.0

# total number of iterations
n_iter = int(np.round(T/dt))
print('Number of interation: ',n_iter)

Number of interation:  1000


In [8]:
# Initialize landscape 
ls = LandscapeWithOcean(NX,NY)

oceanLevelParameter=0.1  # what does this parameter do?
ls.ComputeOceanVolumeFromOceanLevelParameter(Z,NX,NY,oceanLevelParameter)

ls.pool_check(Z,NX,NY)
ls.A = np.zeros((NX,NY))

Minimum elevation           2.1110698683769775
Maximum elevation           21.52335233433031
Beach level                 4.052298114972311
Ocean volume                421.65888211002056
Percentage of ocean surface 10.10204081632653


In [9]:
# Set-up figure
def init_figure():
    fig = plt.figure(figsize=(12.,6.))
    plt.show()
    return fig

def update_figure():
        plt.clf()
        ax1 = fig.add_subplot(121,projection='3d')

        # use equal x-y aspect with an explicit vertical exageration
        vert_exag = 4.
        ax1.set_xlim3d(0,max(NX,NY))
        ax1.set_ylim3d(0,max(NX,NY))
        ax1.set_zlim3d(0,ZMaxOrg)

        ax1.set_title('Surface Relief')

#        surf = ax1.plot_surface(X,Y,Z, cmap = cm.terrain, rstride=1, cstride=1,
#                antialiased=False,linewidth=0)

        ZPlot = np.copy(Z)
        ZPlot[ZPlot<ls.ZBeachLevel] = ls.ZBeachLevel 
        ZPlot -= ls.ZBeachLevel
        ax1.plot_surface(X,Y,ZPlot, cmap = cm.terrain, rstride=1, cstride=1,
                antialiased=False,linewidth=0)

        ax2 = fig.add_subplot(122,aspect='equal')
        ax2.set_title('Elevation')

        #im = ax2.pcolor(Z,cmap=cm.terrain)
        im = ax2.pcolor(ZPlot,cmap=cm.coolwarm)
        cs = ax2.contour(ZPlot,6,colors='k')

        # Add a color bar which maps values to colors.
        cbar = fig.colorbar(im, shrink=0.5, aspect=5)
        # Add the contour line levels to the colorbar
        cbar.add_lines(cs)

        #plt.show()
        plt.draw()
        plt.pause(0.0001)

In [10]:
# Set up figure
fig = init_figure()
update_figure()
Znew = np.copy(Z)

for it in range(1,n_iter+1):
    
    ls.calculate_collection_area(Z,NX,NY)
    ls.A *= dx*dy
    
    for i in range(NX):
        iL = np.mod(i-1,NX) # normally i-1 but observe p.b.c.
        iR = np.mod(i+1,NX) # normally i+1 but observe p.b.c.

        for j in range(NY):
            jD = np.mod(j-1,NY) # normally j-1 but observe p.b.c.
            jU = np.mod(j+1,NY) # normally j+1 but observe p.b.c.
  
            if ls.ocean[i,j]>0:
                Psi_z  = 0;
                Phi_z  = 0;
            else:
                if ls.drain[i,j]>0: #this cell is a drain
                    s1 = (Z[iR,j]  - Z[iL,j] )/(2.*dx)
                    s2 = (Z[i,jU]  - Z[i,jD] )/(2.*dy)
                    s3 = (Z[iR,jD] - Z[iL,jU])/(2. * np.sqrt( dx**2 + dy**2) )
                    s4 = (Z[iR,jU] - Z[iL,jD])/(2. * np.sqrt( dx**2 + dy**2) )
                    gradient = (np.sqrt(s1**2 + s2**2) + np.sqrt(s3**2 + s4**2))/2.
                    Psi_z = K * ( ls.A[i,j]**m * gradient**n - theta_c)            

                elif ls.drainage[i,j]>0: #this cell is a drainage point (it drains a pool)
                
                    if (Z[i,j]>=Z[iR,j]) and ls.pool[iR,j]!=ls.drainage[i,j]: 
                        gradient = (Z[i,j]-Z[iR,j])/dx #pool is on my left, I drain to the right, use this gradiant
                    elif (Z[i,j]>=Z[iL,j]) and ls.pool[iL,j]!=ls.drainage[i,j]:
                        gradient = (Z[i,j]-Z[iL,j])/dx
                    elif (Z[i,j]>=Z[i,jU]) and ls.pool[i,jU]!=ls.drainage[i,j]:
                        gradient = (Z[i,j]-Z[i,jU])/dy
                    elif (Z[i,j]>=Z[i,jD]) and ls.pool[i,jD]!=ls.drainage[i,j]:
                        gradient = (Z[i,j]-Z[i,jD])/dy
                    else:
                        gradient = 0.02 # ??? This does happen (maybe when two pools merge)
                    Psi_z = K * ( ls.A[i,j]**m * gradient**n - theta_c)
                
                else: #this cell is a pool, assume it has some mass diffusion but no erosion!
                    Psi_z = 0.
                
                if (Psi_z<0):
                    Psi_z = 0. 

                # diffusion term
                Phi_z = D * ( (Z[iR,j] - 2.*Z[i,j] + Z[iL,j]) / dx**2 \
                            + (Z[i,jU] - 2.*Z[i,j] + Z[i,jD]) / dx**2 )
           
            Znew[i,j] = Z[i,j] + (Phi_z - Psi_z + uplift )*dt  

            dZdt= (Znew[i,j] - Z[i,j]) / dt
            CFL = abs(dZdt) * dt / min(dx,dy)
            if (CFL>1.0):
                print('\nWarning: Time step of',dt,'is probably too large. Safer would be:',dt/CFL)
            
            if (Znew[i,j]<0.):
                Znew[i,j] = 0. # yes, this does happen at the boundary when kept at zero
    
    #Znew[0,:] = 0.0 # resets front boundary to 0
    Z = np.copy(Znew)
    
    ls.pool_check(Z,NX,NY)

    if (np.mod(it,10)==0): 
        print(it,end='')
        update_figure()
        print(' Ocean level=',ls.ZBeachLevel,' Ocean surface fraction=',100*ls.AOcean/(NX*NY));
    else:
        print('.',end='')

update_figure()
print(' Simulation finished.')


KeyboardInterrupt: 


#1

Upon observation of the starter code, we are building a landscape that comes out of the ocean. This changes things up from the lab since not every part of our map is land, so not every part partakes in the erosion process. The ocean is essentially a large pool however it never leaks out as there is no overflow point.

Ocean level parameter is essentially the sea level. AKA you are at the beach at you are at 0 elevation below sea level. This is the threshold where land becomes land above water as opposed to being underwater.

The ocean level is seeming decreasing after each iteration. Inversely, the ocean surface fraction is increasing. This to me is a little confusing to me as you would expect a direct relation of the two variables however it is saying that as the land erodes into the ocean, the sea starts a little lower since it is being offset by more landmass?



#2: CREATING MY OWN LANDSCAPE


Being from a coastal town, I see lots of erosion occur on the cliffs that sit on the beaches. As time progresses, houses are lost, plots of land become less valuable and people are sometimes crushed. These events can occur more frequent than the erosion that we are seeing because of frequent bluff collapses, constant erosion by waves and poor sedimentary material that cannot withstand rain. However I thought I would craft a hillside/clif of my own and see it erode using the same methods as we have been learning in class.

In [11]:
### Define simulation grid and initial conditions

NX = 35*2 #number of rows
NY = 3*2 #number of columns

d  = 5 # grid spacing in meters
dx = d # keep dx=dy for simplicity
dy = d

LX=NX*dx
LY=NY*dy

In [12]:
### Define simulation grid and initial conditions

fig = init_figure()


# Create a cliff topography: designing a fucntion that fills in data points with a height that fills in a pattern like that of a quadratic equation.
Z = rand(NX,NY)*0.1
for x in range(7,NX):
    for y in range(NY-20):
        if (.5*(x-8))**2 - y < 0:
            Z[x, y] = 800/(.75*(x+10))
        elif ((.5*(x-8))**2 - y > 0) and ((.5*(x-8))**2 - y < 40):
            Z[x, y] = 600/(x)
        else:
            Z[x,y] = 45-x
Z[:7, :] = 62



x = np.arange(NX)
y = np.arange(NY)
X,Y = np.meshgrid(y,x) #strange that y goes first !!!
ZMaxOrg = np.max(Z)
update_figure()

In [13]:
### Physical Parameters
K = 1.0e-6 # meters^(1-2m)/yr

D = 0.005 # m^2/yr

# uplift rate
# uplift = 0.03 / 600.
uplift = .001

### Model parameters

# Time step
dt = d**2 / D / 8. 
#dt = d**2 / D /16. #extra small steps 
print(' dt[years] = ',dt)

#Area exponent A^m, default m=1
m=1

#gradient exponent g^n, default n=1
n=1

#erosion threshold 
theta_c = 10 

 dt[years] =  625.0


In [14]:
# Set up figure
fig = init_figure()
update_figure()
Znew = np.copy(Z)

for it in range(1,n_iter+1):
    
    ls.calculate_collection_area(Z,NX,NY)
    ls.A *= dx*dy
    
    for i in range(NX):
        iL = np.mod(i-1,NX) # normally i-1 but observe p.b.c.
        iR = np.mod(i+1,NX) # normally i+1 but observe p.b.c.

        for j in range(NY):
            jD = np.mod(j-1,NY) # normally j-1 but observe p.b.c.
            jU = np.mod(j+1,NY) # normally j+1 but observe p.b.c.
  
            if ls.ocean[i,j]>0:
                Psi_z  = 0;
                Phi_z  = 0;
            else:
                if ls.drain[i,j]>0: #this cell is a drain
                    s1 = (Z[iR,j]  - Z[iL,j] )/(2.*dx)
                    s2 = (Z[i,jU]  - Z[i,jD] )/(2.*dy)
                    s3 = (Z[iR,jD] - Z[iL,jU])/(2. * np.sqrt( dx**2 + dy**2) )
                    s4 = (Z[iR,jU] - Z[iL,jD])/(2. * np.sqrt( dx**2 + dy**2) )
                    gradient = (np.sqrt(s1**2 + s2**2) + np.sqrt(s3**2 + s4**2))/2.
                    Psi_z = K * ( ls.A[i,j]**m * gradient**n - theta_c)            

                elif ls.drainage[i,j]>0: #this cell is a drainage point (it drains a pool)
                
                    if (Z[i,j]>=Z[iR,j]) and ls.pool[iR,j]!=ls.drainage[i,j]: 
                        gradient = (Z[i,j]-Z[iR,j])/dx #pool is on my left, I drain to the right, use this gradiant
                    elif (Z[i,j]>=Z[iL,j]) and ls.pool[iL,j]!=ls.drainage[i,j]:
                        gradient = (Z[i,j]-Z[iL,j])/dx
                    elif (Z[i,j]>=Z[i,jU]) and ls.pool[i,jU]!=ls.drainage[i,j]:
                        gradient = (Z[i,j]-Z[i,jU])/dy
                    elif (Z[i,j]>=Z[i,jD]) and ls.pool[i,jD]!=ls.drainage[i,j]:
                        gradient = (Z[i,j]-Z[i,jD])/dy
                    else:
                        gradient = 0.02 # ??? This does happen (maybe when two pools merge)
                    Psi_z = K * ( ls.A[i,j]**m * gradient**n - theta_c)
                
                else: #this cell is a pool, assume it has some mass diffusion but no erosion!
                    Psi_z = 0.
                
                if (Psi_z<0):
                    Psi_z = 0. 

                # diffusion term
                Phi_z = D * ( (Z[iR,j] - 2.*Z[i,j] + Z[iL,j]) / dx**2 \
                            + (Z[i,jU] - 2.*Z[i,j] + Z[i,jD]) / dx**2 )
           
            Znew[i,j] = Z[i,j] + (Phi_z - Psi_z + uplift )*dt  

            dZdt= (Znew[i,j] - Z[i,j]) / dt
            CFL = abs(dZdt) * dt / min(dx,dy)
            if (CFL>1.0):
                print('\nWarning: Time step of',dt,'is probably too large. Safer would be:',dt/CFL)
            
            if (Znew[i,j]<0.):
                Znew[i,j] = 0. # yes, this does happen at the boundary when kept at zero
    
    #Znew[0,:] = 0.0 # resets front boundary to 0
    Z = np.copy(Znew)
    
    ls.pool_check(Z,NX,NY)

    if (np.mod(it,10)==0): 
        print(it,end='')
        update_figure()
        print(' Ocean level=',ls.ZBeachLevel,' Ocean surface fraction=',100*ls.AOcean/(NX*NY));
    else:
        print('.',end='')

update_figure()
print(' Simulation finished.')

IndexError: index 69 is out of bounds for axis 1 with size 6

#3:
An increase in D will smooth out the erosion of land it seems. I upped it by a magnitude of 10 and it produced a very appealing slope and the 2d topographical map was very smooth/rounded.

An increase in uplift seemingly repopulated the ocean with landmass. This doesnt seem to realistic, however it did make a nice chanel which remebeled stream flow path down the side of the cliff when uplift was equal to .5

the m affects the weight of the current "drainage area" associated with the erosion process, increasing m would then make the system much more conformed(not much diversity)

The n affects the weight of the gradiant in the erosion model. An increase would make it converge erode towards the lower land much faster tahn the higher land.

For both the n and the m, a decrease would lead to the inverse result as described above.
